In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 假设你的数据集路径是 'your_dataset.csv'
# 加载数据集
ds = pd.read_csv('3_clean.csv')
data = ds[:10000]
# 预览数据
print(data.head())

# 获取描述性统计信息
print(data.describe())

In [ ]:

# 分布可视化
# 账户数量的分布
sns.histplot(data['no_of_accounts_with_syf_13_march'], kde=True)
plt.title('Distribution of Number of Accounts')
plt.xlabel('Number of Accounts with SYF 13 March')
plt.ylabel('Frequency')
plt.show()

# 解决状态的分布
sns.countplot(x='resolved', data=data)
plt.title('Distribution of Resolved Status')
plt.xlabel('Resolved Status')
plt.ylabel('Count')
plt.show()

In [ ]:

# 分析账户数量和解决状态之间的关系
sns.boxplot(x='resolved', y='no_of_accounts_with_syf_13_march', data=data)
plt.title('Relationship between Number of Accounts and Resolved Status')
plt.xlabel('Resolved Status')
plt.ylabel('Number of Accounts with SYF 13 March')
plt.show()